In [1]:
from ipywidgets import IntProgress
from IPython.display import display,HTML,clear_output
import random
import time
import math
import ROOT
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from uits3_krakow22.src.Event import Event
from uits3_krakow22.src.Cluster import Cluster
from uits3_krakow22.src.Track import Track
from skspatial.plotting import plot_3d
from skspatial.objects import Points, Cylinder, Plane

import tensorflow as tf

display(HTML("<style>.container { width:95% !important; }</style>"))
display(HTML("<style>table {float:left;}</style>"))

Welcome to JupyROOT 6.26/10


2023-06-12 13:54:31.036561: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-12 13:54:31.190402: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-12 13:54:31.191503: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-12 13:54:32.715779: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Simulation
#### Define the target

In [2]:
class Fiber:
    def __init__(self,center=[0,0,0],radius=1,height=10):
        self.center = center
        self.radius = radius
        self.height = height
    
    def getRandomPoint(self):
        R = random.random()*self.radius
        height = (random.random()-0.5)*self.height
        theta = random.random()*2*math.pi
        coordinate = [self.center[0]+R*math.cos(theta),self.center[1]+height,self.center[2]+R*math.sin(theta)]
        return coordinate
    
dist = 0.5
origin = [[0,0,0]]
row1 = [[0,0,-2*dist*math.sqrt(3)]]
row2 = [[-dist*0.5,0,-dist*(3*math.sqrt(3)/2)],[dist*0.5,0,-dist*(3*math.sqrt(3)/2)]]
row3 = [[-dist,0,-dist*math.sqrt(3)],[0,0,-dist*math.sqrt(3)],[dist,0,-dist*math.sqrt(3)]]
row4 = [[-dist*1.5,0,-dist*math.sqrt(3)/2],[-dist*0.5,0,-dist*math.sqrt(3)/2],[dist*0.5,0,-dist*math.sqrt(3)/2],[dist*1.5,0,-dist*math.sqrt(3)/2]]
centers = origin + row1 + row2 + row3 + row4

for center in centers:
    center[2] = center[2]-16.83
print(centers)

fibers = [Fiber(center=center,height=6,radius=0.075) for center in centers]

[[0, 0, -16.83], [0, 0, -18.562050807568877], [-0.25, 0, -18.129038105676656], [0.25, 0, -18.129038105676656], [-0.5, 0, -17.696025403784436], [0, 0, -17.696025403784436], [0.5, 0, -17.696025403784436], [-0.75, 0, -17.26301270189222], [-0.25, 0, -17.26301270189222], [0.25, 0, -17.26301270189222], [0.75, 0, -17.26301270189222]]


#### Create tracks and calculate clusters

In [3]:
def simulateClusters(reps,displacement=None):
    if not displacement: displacement = {"ALPIDE_0" : [0,0,0],"ALPIDE_1" : [0,0,0],"ALPIDE_2" : [0,0,0],"ALPIDE_3" : [0,0,0],"ALPIDE_4" : [0,0,0]}
    
    localPositions = []
    
    openingAngle = (87.08/360)*2*math.pi
    
    # detector geometry
    ALPIDE_0 = Cylinder([0,0,12.5-7.5], [0,0,15], 30)
    ALPIDE_1 = Cylinder([0,0,6.25-7.5], [0,0,15], 24)
    ALPIDE_2 = Cylinder([0,0,0-7.5], [0,0,15], 18)
    ALPIDE_3 = Cylinder([0,0,12.5-7.5], [0,0,15], 30)
    ALPIDE_4 = Cylinder([0,0,0-7.5], [0,0,15], 18)
    alpidesLeft = [ALPIDE_3,ALPIDE_4]
    alpidesRight = [ALPIDE_0,ALPIDE_1,ALPIDE_2]
    
    for i in range(reps):
        localTmp=[]
        fiber = random.choice(fibers)
        phi = math.pi*(0.5+(random.random()-0.5)*0.2)#*0.265258238
        theta = math.pi/2-openingAngle/2
        shift = (random.random()-0.5)*0.1
        origin = fiber.getRandomPoint()
       
        #calculate right arm clusters
        trackRight = Track(point=origin,vector=[-math.sin(phi)*math.cos(-theta+shift), -math.cos(phi), -math.sin(phi)*math.sin(-theta+shift)])
        for alpide in alpidesRight:
            if alpide is ALPIDE_0: detector = "ALPIDE_0"
            elif alpide is ALPIDE_1: detector = "ALPIDE_1"
            elif alpide is ALPIDE_2: detector = "ALPIDE_2"
            intersection = alpide.intersect_line(trackRight.line)
            cluster = Cluster()
            cluster.setPositionGlobal(intersection[1])
            cluster.alignLocal(displacement.get(detector))
            localTmp.extend(cluster.localPos[0:2])
        
        #calculate left arm clusters
        trackLeft = Track(point= origin,vector=[math.sin(phi)*math.cos(theta+shift), math.cos(phi), math.sin(phi)*math.sin(theta+shift)])
        for alpide in alpidesLeft:
            if alpide is ALPIDE_3: detector = "ALPIDE_3"
            elif alpide is ALPIDE_4: detector = "ALPIDE_4"
            intersection = alpide.intersect_line(trackLeft.line)
            cluster = Cluster()
            cluster.setPositionGlobal(intersection[1])
            cluster.alignLocal(displacement.get(detector))
            localTmp.extend(cluster.localPos[0:2])
            
        localPositions.append(localTmp)
    return localPositions

def dispMap2Array(displacement):
    output=[]
    for key in ["ALPIDE_0","ALPIDE_1","ALPIDE_2","ALPIDE_3","ALPIDE_4"]:
        output.extend(displacement[key])
    return output

In [54]:
def split_train_val_indices(data_size, val_ratio=0.2, shuffle=True, random_seed=None):
    """
    Randomly splits the indices into training and validation sets.

    Args:
        data_size (int): Total number of data points.
        val_ratio (float): Ratio of validation data points (default: 0.2).
        shuffle (bool): Whether to shuffle the indices (default: True).
        random_seed (int): Random seed for reproducibility (default: None).

    Returns:
        train_indices (ndarray): Array of indices for the training set.
        val_indices (ndarray): Array of indices for the validation set.
    """
    # Set the random seed for reproducibility
    if random_seed is not None:
        np.random.seed(random_seed)

    # Create an array of indices
    indices = np.arange(data_size)

    # Shuffle the indices if required
    if shuffle:
        np.random.shuffle(indices)

    # Calculate the number of validation samples
    val_size = int(val_ratio * data_size)

    # Split the indices into training and validation sets
    train_indices = indices[val_size:]
    val_indices = indices[:val_size]

    return train_indices, val_indices

### Generate simulated data

In [108]:
data = []
misal = []
for disp in np.linspace(-1, 1, num=25):
    displacement = {
        "ALPIDE_0" : [0,0,0],
        "ALPIDE_1" : [disp,0,0],
        "ALPIDE_2" : [0,0,0],
        "ALPIDE_3" : [0,0,0],
        "ALPIDE_4" : [0,0,0]
    }
    nReps4setting = 2000
    data.extend(simulateClusters(nReps4setting,displacement))
    misal.extend([dispMap2Array(displacement)]*nReps4setting)
input_data = np.array(data)
input_data = np.reshape(input_data, (-1, 10))
misalignments  = np.array(misal)

## Machine learning with tensorflow

In [107]:
def trainNetwork(input_data,misalignments,units=64,nLayers=1,nEpochs=10,batch=32):
    data_size = len(input_data)

    # Normalize the input data
    #input_data = (input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)

    # Split the indices into training and validation sets
    train_indices, val_indices = split_train_val_indices(data_size, val_ratio=0.2, shuffle=True, random_seed=42)
    train_input = input_data[train_indices]
    train_misalignments = misalignments[train_indices]
    val_input = input_data[val_indices]
    val_misalignments = misalignments[val_indices]

    # Reshape the input data to have at least two dimensions
    train_input = np.reshape(train_input, (-1, 10))
    val_input = np.reshape(val_input, (-1, 10))

    # Create the sequential model
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units=units, activation='relu', input_shape=(10,)))
    
    for i in range(nLayers-1):
        model.add(tf.keras.layers.Dense(units=units, activation='relu'))

    # Add the output layer with the desired number of units (30 in your case)
    model.add(tf.keras.layers.Dense(units=15))

    # Compile the model
    model.compile(optimizer='adam', loss='mse')

    # Train the model
    num_epochs = nEpochs
    batch_size = batch
    model.fit(x=train_input, y=train_misalignments, epochs=num_epochs, batch_size=batch_size, validation_data=(val_input, val_misalignments))
    return model

In [111]:
def evalModel(model,displacement,nReps=1000):
    data = []
    data.extend(simulateClusters(nReps,displacement))
    test_data = np.reshape(np.array(data), (-1, 10))
    predictions = model.predict(test_data)
    predicted_disp = np.mean(predictions,axis=0)
    dist = np.linalg.norm(dispMap2Array(displacement)-predicted_disp)
    return dist

In [112]:
disp2test = {
        "ALPIDE_0" : [0,0,0],
        "ALPIDE_1" : [-0.456,0,0],
        "ALPIDE_2" : [0,0,0],
        "ALPIDE_3" : [0,0,0],
        "ALPIDE_4" : [0,0,0]
    }
model = trainNetwork(input_data,misalignments)
evalModel(model,disp2test)

Epoch 1/10
1250/1250 [==============================] - 1s 837us/step - loss: 0.0632 - val_loss: 0.0034
Epoch 2/10
1250/1250 [==============================] - 1s 791us/step - loss: 0.0016 - val_loss: 7.7024e-04
Epoch 3/10
1250/1250 [==============================] - 1s 791us/step - loss: 5.1446e-04 - val_loss: 3.2435e-04
Epoch 4/10
1250/1250 [==============================] - 1s 791us/step - loss: 2.5322e-04 - val_loss: 1.7284e-04
Epoch 5/10
1250/1250 [==============================] - 1s 797us/step - loss: 1.7481e-04 - val_loss: 1.6746e-04
Epoch 6/10
1250/1250 [==============================] - 1s 798us/step - loss: 1.5094e-04 - val_loss: 1.0386e-04
Epoch 7/10
1250/1250 [==============================] - 1s 783us/step - loss: 1.2695e-04 - val_loss: 6.5924e-05
Epoch 8/10
1250/1250 [==============================] - 1s 805us/step - loss: 1.0731e-04 - val_loss: 7.0026e-05
Epoch 9/10
1250/1250 [==============================] - 1s 795us/step - loss: 1.3503e-04 - val_loss: 1.8123e-04
Epoc

0.010515563089260731